## LEFTOVERS TIME

Here are a bunch of cells I don't want in the final notebook, but don't want to get rid of completely

### Tune with sklearn API

Skip this for now, it is too much of a pain. Maybe try it again once I update things, but I need to stop spending time on this guy.

It has memory leaks, it won't do the callback, last error was supremely unhelpful "an actor died".

In [9]:
from ray.tune.integration.xgboost import TuneReportCheckpointCallback
# Using this as a reference for the sklearn API way where I do callback stuff myself
# https://docs.ray.io/en/releases-1.11.0/tune/tutorials/tune-xgboost.html
dtrain = xgbr.RayDMatrix(X_train, y_train)
dtest = xgbr.RayDMatrix(X_test, y_test)
def my_trainer(config):
    evals_result = {}
    xgbr_model = xgbr.RayXGBRegressor(**config)
    #    n_jobs=3,  # In XGBoost-Ray, n_jobs sets the number of actors
    #    random_state=42,
    #    objective='reg:tweedie',
        #use_label_encoder=False,
        #eval_metric=mean_poisson_deviance
    #)
    xgbr_model.fit(dtrain, None, ray_params=ray_param, eval_metric="rmse", callbacks=[TuneReportCheckpointCallback])
    predictions = xgbr_model.predict(X_test) 
    actuals = y_test
    score = mean_poisson_deviance(predictions, actuals)
    tune.report(score=score)
    #xgbr_model.save_model("model.xgb")

config = {
    #"tree_method": "approx",
    "random_state": 42,
    "objective": "reg:gamma",
    #"objective": "reg:squarederror",
    #"eval_metric": "rmse",
    "eta": tune.loguniform(3e-3, 3e-1),
    "max_depth": tune.randint(4, 8)
}

analysis = tune.run(
    my_trainer,
    config=config,
    local_dir=tune_results_dir, # gotta change this to avoid permission denied on ~/ray_results
    sync_config=tune.SyncConfig(syncer=None), # gotta change this to avoid weird "getpwuid(): uid not found: 12574"
    metric="eval-rmse", # need a serious digression here on how to find the errors
    mode="min",
    num_samples=4,
    resources_per_trial=ray_param.get_tune_resources(),
    verbose=1,
    progress_reporter=tune.JupyterNotebookReporter(overwrite=True)
)

RayTaskError(TuneError): [36mray::run()[39m (pid=465, ip=10.0.106.182)
  File "/home/ray/anaconda3/lib/python3.8/site-packages/ray/tune/tune.py", line 620, in run
    raise TuneError("Trials did not complete", incomplete_trials)
ray.tune.error.TuneError: ('Trials did not complete', [my_trainer_6a544_00000, my_trainer_6a544_00001, my_trainer_6a544_00002, my_trainer_6a544_00003])

# Minimal Example (old iris one)
Use Iris example to validate packages etc.

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [2]:
from sklearn.metrics import confusion_matrix
import xgboost as xgb

# Play with some of these params:
# https://xgboost.readthedocs.io/en/stable/python/python_api.html#xgboost.XGBClassifier
xgb_model = xgb.XGBClassifier(
    n_jobs=1,
    random_state=42,
    use_label_encoder=False
).fit(X_train, y_train) 
predictions = xgb_model.predict(X_test) 
actuals = y_test
print(confusion_matrix(actuals, predictions)) 


[16:07:23] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[[19  0  0]
 [ 0 13  0]
 [ 0  0 13]]


In [3]:
import ray
import os

if ray.is_initialized() == False:
    service_host = os.environ["RAY_HEAD_SERVICE_HOST"]
    service_port = os.environ["RAY_HEAD_SERVICE_PORT"]
    ray.init(f"ray://{service_host}:{service_port}")

In [4]:
import xgboost_ray as xgbray
train_set = xgbray.RayDMatrix(X_train, y_train)


In [16]:
xgbray_model = xgbray.RayXGBClassifier(
    n_jobs=3,  # In XGBoost-Ray, n_jobs sets the number of actors
    random_state=42,
    use_label_encoder=False
)


In [17]:
# This can result in an error:
#  "ModuleNotFoundError: No module named 'tblib'"
# if tblib is not installed on the workspace environment
xgbray_model.fit(X_train, y_train)

(_wrapped pid=219) 2022-10-16 09:21:58,045	INFO main.py:926 -- [RayXGBoost] Created 3 new actors (3 total actors). Waiting until actors are ready for training.
(_RemoteRayXGBoostActor pid=77, ip=10.0.59.16) [09:21:59] task [xgboost.ray]:140375377122976 got new rank 2
(_wrapped pid=219) 2022-10-16 09:21:59,653	INFO main.py:971 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=521) [09:21:59] task [xgboost.ray]:139886842213760 got new rank 0
(_RemoteRayXGBoostActor pid=201, ip=10.0.42.51) [09:21:59] task [xgboost.ray]:140319019763360 got new rank 1
(_RemoteRayXGBoostActor pid=201, ip=10.0.42.51) [09:21:59] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
(_RemoteRayXGBoostActor pid=77, ip=10.0.59.16) [09:21:59] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the def

RayXGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                 colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
                 importance_type='gain', interaction_constraints='',
                 learning_rate=0.300000012, max_delta_step=0, max_depth=6,
                 min_child_weight=1, missing=nan, monotone_constraints='()',
                 n_estimators=100, n_jobs=3, num_parallel_tree=1,
                 objective='multi:softprob', random_state=42, reg_alpha=0,
                 reg_lambda=1, scale_pos_weight=None, subsample=1,
                 tree_method='approx', use_label_encoder=False,
                 validate_parameters=1, verbosity=None)

In [18]:
# This can result in an error:
#  "TypeError: predict() got an unexpected keyword argument 'iteration_range'"
#  if xgboost does not match across workspace and cluster
#  (seen with workspace xgboost 1.5.2 and cluster xgboost 1.3.3)
pred_ray = xgbray_model.predict(X_test)

(predict pid=219) 2022-10-16 09:22:27,175	INFO main.py:1490 -- [RayXGBoost] Created 3 remote actors.


In [19]:
pred_ray

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0,
       0])

In [20]:
confusion_matrix(actuals, pred_ray)

array([[19,  0,  0],
       [ 0, 13,  0],
       [ 0,  0, 13]])

In [21]:
xgbray_model.save_model("model.xgb")

A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: ffffffffffffffffbd06aeef78adce52ce0d5ef401000000 Worker ID: 2aa6b173d1acc23d4a112316250fea4fe77560efaed672ef02676859 Node ID: 1f1a7582775718123e59a0aecba0aac17958a709e6c9d325c2974922 Worker IP address: 10.0.113.219 Worker port: 10011 Worker PID: 582
(predict pid=219) 2022-10-16 09:22:30,565	INFO main.py:1507 -- [RayXGBoost] Starting XGBoost prediction.


In [22]:
xgbray_model = xgb.Booster(model_file="model.xgb")

In [23]:
pred_ray_2 = xgbray.predict(xgbray_model, xgbray.RayDMatrix(X_test, y_test),ray_params=xgbray.RayParams(num_actors=2))
print(pred_ray_2)


(predict pid=219) 2022-10-16 09:22:51,347	INFO main.py:1490 -- [RayXGBoost] Created 2 remote actors.


[[2.24131113e-03 9.88115311e-01 9.64334048e-03]
 [9.83814895e-01 1.48946904e-02 1.29038270e-03]
 [4.66208701e-04 2.38856534e-03 9.97145236e-01]
 [1.86032848e-03 9.86977160e-01 1.11625614e-02]
 [3.16508673e-03 9.68412161e-01 2.84227841e-02]
 [9.92588520e-01 6.10954128e-03 1.30189036e-03]
 [5.25772432e-03 9.93141353e-01 1.60099554e-03]
 [2.31618597e-03 1.23781925e-02 9.85305607e-01]
 [1.21991476e-03 9.54050660e-01 4.47293557e-02]
 [1.92491431e-03 9.97014761e-01 1.06028316e-03]
 [3.55823454e-03 1.90159660e-02 9.77425754e-01]
 [9.95287895e-01 3.28664947e-03 1.42548338e-03]
 [9.88850653e-01 9.85232182e-03 1.29698764e-03]
 [9.95393991e-01 3.28699965e-03 1.31905486e-03]
 [9.95407343e-01 3.28704366e-03 1.30558747e-03]
 [1.23849756e-03 9.92311537e-01 6.44996716e-03]
 [4.24794620e-04 1.08198158e-03 9.98493195e-01]
 [2.42856308e-03 9.94871795e-01 2.69956584e-03]
 [4.18672757e-03 9.91830409e-01 3.98287782e-03]
 [2.70034914e-04 6.27312518e-04 9.99102712e-01]
 [9.95407343e-01 3.28704366e-03 1.305587

(predict pid=219) 2022-10-16 09:22:54,253	INFO main.py:1507 -- [RayXGBoost] Starting XGBoost prediction.


In [13]:
import numpy as np
np.argmax(pred_ray_2, axis=1)

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0,
       0])

In [14]:
confusion_matrix(actuals, np.argmax(pred_ray_2, axis=1))

array([[19,  0,  0],
       [ 0, 13,  0],
       [ 0,  0, 13]])

In [15]:
# sanity check the probabilities
np.sum(pred_ray_2, axis=1)

array([0.99999994, 0.99999994, 1.        , 1.        , 1.        ,
       0.99999994, 1.0000001 , 1.        , 0.99999994, 1.        ,
       0.99999994, 1.        , 1.        , 1.0000001 , 0.99999994,
       1.        , 1.        , 0.99999994, 1.        , 1.        ,
       0.99999994, 1.        , 0.99999994, 1.        , 0.99999994,
       0.99999994, 1.        , 0.99999994, 1.        , 1.0000001 ,
       0.99999994, 0.99999994, 1.0000001 , 0.99999994, 0.99999994,
       0.99999994, 1.0000001 , 1.        , 0.99999994, 1.        ,
       0.99999994, 0.99999994, 1.        , 0.99999994, 0.99999994],
      dtype=float32)